In [507]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import itertools
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.model_selection import KFold

In [508]:
# Define the number of rows for the synthetic dataset
num_rows = 2421

# Generate synthetic data for each column
order_date = [datetime(2009, 1, 1) + timedelta(days=np.random.randint(730)) for _ in range(num_rows)]
requested_delivery_date = [date + timedelta(days=np.random.randint(30)) for date in order_date]
country_codes = np.random.choice(['US', 'UK', 'DE', 'FR', 'IT'], size=num_rows)
product_codes = np.random.choice(['P001', 'P002', 'P003'], size=num_rows)
descriptions = [f'Description_{i}' for i in range(1, num_rows + 1)]
order_types = np.random.choice(['VO', 'Non-VO'], size=num_rows)
customer_order_codes = np.random.randint(1000, 2000, size=num_rows)
values = np.random.uniform(50, 500, size=num_rows)
currencies = np.random.choice(['USD', 'EUR', 'GBP'], size=num_rows)

items = np.random.randint(1, 800, size=num_rows)

routes = np.random.choice(['Route_A', 'Route_B', 'Route_C'], size=num_rows)

requested_delivery_date = []
for i in range(num_rows):
    if order_types[i] == 'VO':
        requested_delivery_date.append(order_date[i] + timedelta(days=np.random.randint(27, 34)))
    else:
        requested_delivery_date.append(order_date[i] + timedelta(days=np.random.randint(30)))

In [509]:

# Create a DataFrame with the generated data
data = pd.DataFrame({
    'Order Date': order_date,
    'Requested Delivery Date': requested_delivery_date,
    'Customer Country Code': country_codes,
    'Product Code': product_codes,
    'Description': descriptions,
    'Order type': order_types,
    'Customer Order Code': customer_order_codes,
    'Value': values,
    'Currency': currencies,
    'Items': items,
    'Route': routes
})

In [510]:
data

,Order Date,Requested Delivery Date,Customer Country Code,Product Code,Description,Order type,Customer Order Code,Value,Currency,Items,Route
0,2010-03-28,2010-04-16,FR,P002,Description_1,Non-VO,1854,382.706394,EUR,492,Route_C
1,2010-12-22,2011-01-12,DE,P003,Description_2,Non-VO,1634,223.874999,USD,510,Route_B
2,2009-02-10,2009-03-09,UK,P003,Description_3,VO,1350,305.198381,GBP,764,Route_A
3,2010-12-15,2010-12-22,US,P002,Description_4,Non-VO,1748,59.290286,EUR,486,Route_C
4,2010-03-22,2010-03-31,FR,P002,Description_5,Non-VO,1341,288.235547,GBP,316,Route_B
...,...,...,...,...,...,...,...,...,...,...,...
2416,2009-11-30,2009-12-15,FR,P002,Description_2417,Non-VO,1476,184.410110,EUR,622,Route_A
2417,2009-06-12,2009-06-27,FR,P001,Description_2418,Non-VO,1320,86.185526,GBP,341,Route_B
2418,2010-05-23,2010-06-22,DE,P002,Description_2419,VO,1734,127.730830,USD,321,Route_B
2419,2009-04-05,2009-05-04,IT,P002,Description_2420,VO,1398,188.441904,GBP,732,Route_B


In [511]:
data['MOY_Requested Delivery Date'] = data['Requested Delivery Date'].dt.month

In [512]:
data

,Order Date,Requested Delivery Date,Customer Country Code,Product Code,Description,Order type,Customer Order Code,Value,Currency,Items,Route,MOY_Requested Delivery Date
0,2010-03-28,2010-04-16,FR,P002,Description_1,Non-VO,1854,382.706394,EUR,492,Route_C,4
1,2010-12-22,2011-01-12,DE,P003,Description_2,Non-VO,1634,223.874999,USD,510,Route_B,1
2,2009-02-10,2009-03-09,UK,P003,Description_3,VO,1350,305.198381,GBP,764,Route_A,3
3,2010-12-15,2010-12-22,US,P002,Description_4,Non-VO,1748,59.290286,EUR,486,Route_C,12
4,2010-03-22,2010-03-31,FR,P002,Description_5,Non-VO,1341,288.235547,GBP,316,Route_B,3
...,...,...,...,...,...,...,...,...,...,...,...,...
2416,2009-11-30,2009-12-15,FR,P002,Description_2417,Non-VO,1476,184.410110,EUR,622,Route_A,12
2417,2009-06-12,2009-06-27,FR,P001,Description_2418,Non-VO,1320,86.185526,GBP,341,Route_B,6
2418,2010-05-23,2010-06-22,DE,P002,Description_2419,VO,1734,127.730830,USD,321,Route_B,6
2419,2009-04-05,2009-05-04,IT,P002,Description_2420,VO,1398,188.441904,GBP,732,Route_B,5


In [513]:
exchange_rates = {'USD': 1.26, 'EUR': 1.48}

In [514]:
data['Value_CAD'] = data.apply(lambda row: row['Value'] * exchange_rates.get(row['Currency'], 1), axis=1)


In [515]:
data['Total_Price'] = data['Value_CAD'] * data['Items']

In [516]:
agg_data = data.groupby(['Product Code', 'MOY_Requested Delivery Date']).agg({'Total_Price': 'sum', 'Items': 'sum'}).reset_index()

In [517]:
agg_data['Average_Price'] = agg_data['Total_Price'] / agg_data['Items']

In [518]:
label_encoder = LabelEncoder()
agg_data['Product Code'] = label_encoder.fit_transform(agg_data['Product Code']) + 1

In [519]:
df_choice=agg_data.drop(['Total_Price'], axis=1)

In [520]:
X = df_choice.drop(['Product Code'], axis=1)
Y = df_choice['Product Code']

In [521]:
X

,MOY_Requested Delivery Date,Items,Average_Price
0,1,29798,345.787470
1,2,23080,356.820775
2,3,26428,303.017557
3,4,23712,358.449826
4,5,34719,365.152850
5,6,27533,392.939217
6,7,28774,336.888834
7,8,28657,313.052387
8,9,27499,352.316930
9,10,30394,317.158946


In [522]:
X = sm.add_constant(X)

In [523]:
X

,const,MOY_Requested Delivery Date,Items,Average_Price
0,1.0,1,29798,345.787470
1,1.0,2,23080,356.820775
2,1.0,3,26428,303.017557
3,1.0,4,23712,358.449826
4,1.0,5,34719,365.152850
5,1.0,6,27533,392.939217
6,1.0,7,28774,336.888834
7,1.0,8,28657,313.052387
8,1.0,9,27499,352.316930
9,1.0,10,30394,317.158946


In [524]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [525]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [526]:
model = sm.MNLogit(Y_train, X_train)
result = model.fit(ref_cat=1)

Optimization terminated successfully.
         Current function value: 1.041933
         Iterations 5


c:\Users\tn_ad\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\optimizer.py:18: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol, ridge_factor. The list of unsupported keyword arguments passed include: ref_cat. After release 0.14, this will raise.
  warnings.warn(


In [527]:
print(result.summary())

                          MNLogit Regression Results                          
Dep. Variable:           Product Code   No. Observations:                   28
Model:                        MNLogit   Df Residuals:                       20
Method:                           MLE   Df Model:                            6
Date:                Sat, 09 Dec 2023   Pseudo R-squ.:                 0.03435
Time:                        22:50:23   Log-Likelihood:                -29.174
converged:                       True   LL-Null:                       -30.212
Covariance Type:            nonrobust   LLR p-value:                    0.9126
             Product Code=2       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                           2.5957      7.627      0.340      0.734     -12.353      17.544
MOY_Requested Delivery Date     0.0715      0.144      0.498      0.618      -0.

In [528]:
Y_prob = result.predict(X_test_scaled)

In [529]:
print(Y_prob)

[[0.3264749  0.36555242 0.30797269]
 [0.33747406 0.30545208 0.35707386]
 [0.33643235 0.31078646 0.35278119]
 [0.33251462 0.33445897 0.33302641]
 [0.33579488 0.32523237 0.33897275]
 [0.33128249 0.34024243 0.32847509]
 [0.32922072 0.35341137 0.31736791]
 [0.33907169 0.30124533 0.35968298]]


In [530]:
pip install pmdarima

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [531]:
from pmdarima import auto_arima
from sklearn.model_selection import TimeSeriesSplit

In [532]:
data

,Order Date,Requested Delivery Date,Customer Country Code,Product Code,Description,Order type,Customer Order Code,Value,Currency,Items,Route,MOY_Requested Delivery Date,Value_CAD,Total_Price
0,2010-03-28,2010-04-16,FR,P002,Description_1,Non-VO,1854,382.706394,EUR,492,Route_C,4,566.405464,278671.488205
1,2010-12-22,2011-01-12,DE,P003,Description_2,Non-VO,1634,223.874999,USD,510,Route_B,1,282.082499,143862.074344
2,2009-02-10,2009-03-09,UK,P003,Description_3,VO,1350,305.198381,GBP,764,Route_A,3,305.198381,233171.563227
3,2010-12-15,2010-12-22,US,P002,Description_4,Non-VO,1748,59.290286,EUR,486,Route_C,12,87.749623,42646.316813
4,2010-03-22,2010-03-31,FR,P002,Description_5,Non-VO,1341,288.235547,GBP,316,Route_B,3,288.235547,91082.432974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2416,2009-11-30,2009-12-15,FR,P002,Description_2417,Non-VO,1476,184.410110,EUR,622,Route_A,12,272.926963,169760.571245
2417,2009-06-12,2009-06-27,FR,P001,Description_2418,Non-VO,1320,86.185526,GBP,341,Route_B,6,86.185526,29389.264286
2418,2010-05-23,2010-06-22,DE,P002,Description_2419,VO,1734,127.730830,USD,321,Route_B,6,160.940846,51662.011464
2419,2009-04-05,2009-05-04,IT,P002,Description_2420,VO,1398,188.441904,GBP,732,Route_B,5,188.441904,137939.473919


In [533]:
data['Requested Delivery Date'] = pd.to_datetime(data['Requested Delivery Date'])
data['Delivery Month'] = (
    (data['Requested Delivery Date'].dt.year - data['Requested Delivery Date'].dt.year.min()) * 12 +
    data['Requested Delivery Date'].dt.month
)

In [534]:
data

,Order Date,Requested Delivery Date,Customer Country Code,Product Code,Description,Order type,Customer Order Code,Value,Currency,Items,Route,MOY_Requested Delivery Date,Value_CAD,Total_Price,Delivery Month
0,2010-03-28,2010-04-16,FR,P002,Description_1,Non-VO,1854,382.706394,EUR,492,Route_C,4,566.405464,278671.488205,16
1,2010-12-22,2011-01-12,DE,P003,Description_2,Non-VO,1634,223.874999,USD,510,Route_B,1,282.082499,143862.074344,25
2,2009-02-10,2009-03-09,UK,P003,Description_3,VO,1350,305.198381,GBP,764,Route_A,3,305.198381,233171.563227,3
3,2010-12-15,2010-12-22,US,P002,Description_4,Non-VO,1748,59.290286,EUR,486,Route_C,12,87.749623,42646.316813,24
4,2010-03-22,2010-03-31,FR,P002,Description_5,Non-VO,1341,288.235547,GBP,316,Route_B,3,288.235547,91082.432974,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2416,2009-11-30,2009-12-15,FR,P002,Description_2417,Non-VO,1476,184.410110,EUR,622,Route_A,12,272.926963,169760.571245,12
2417,2009-06-12,2009-06-27,FR,P001,Description_2418,Non-VO,1320,86.185526,GBP,341,Route_B,6,86.185526,29389.264286,6
2418,2010-05-23,2010-06-22,DE,P002,Description_2419,VO,1734,127.730830,USD,321,Route_B,6,160.940846,51662.011464,18
2419,2009-04-05,2009-05-04,IT,P002,Description_2420,VO,1398,188.441904,GBP,732,Route_B,5,188.441904,137939.473919,5


In [535]:
demand = data.groupby(['Order type', 'Delivery Month']).agg({'Total_Price': 'sum', 'Items': 'sum'}).reset_index()

In [536]:
demand

,Order type,Delivery Month,Total_Price,Items
0,Non-VO,1,4.589957e+06,12266
1,Non-VO,2,5.256095e+06,15144
2,Non-VO,3,5.825314e+06,16621
3,Non-VO,4,6.631700e+06,22263
4,Non-VO,5,9.619035e+06,27069
5,Non-VO,6,5.130315e+06,12274
6,Non-VO,7,6.560863e+06,19534
7,Non-VO,8,4.053775e+06,14961
8,Non-VO,9,5.903531e+06,17394
9,Non-VO,10,8.228974e+06,21112


In [537]:
demand['Advance_Order'] = (demand['Order type'] == 'VO') * demand['Items']
demand['Urgent_Order'] = (demand['Order type'] == 'Non-VO') * demand['Items']

In [538]:
demand

,Order type,Delivery Month,Total_Price,Items,Advance_Order,Urgent_Order
0,Non-VO,1,4.589957e+06,12266,0,12266
1,Non-VO,2,5.256095e+06,15144,0,15144
2,Non-VO,3,5.825314e+06,16621,0,16621
3,Non-VO,4,6.631700e+06,22263,0,22263
4,Non-VO,5,9.619035e+06,27069,0,27069
5,Non-VO,6,5.130315e+06,12274,0,12274
6,Non-VO,7,6.560863e+06,19534,0,19534
7,Non-VO,8,4.053775e+06,14961,0,14961
8,Non-VO,9,5.903531e+06,17394,0,17394
9,Non-VO,10,8.228974e+06,21112,0,21112


In [539]:
demand = demand.groupby(['Delivery Month']).agg({'Total_Price': 'sum', 'Advance_Order': 'sum', 'Urgent_Order': 'sum'}).reset_index()
demand['Average_Price'] = demand['Total_Price'] /(demand['Advance_Order']+demand['Urgent_Order'])

In [540]:
demand

,Delivery Month,Total_Price,Advance_Order,Urgent_Order,Average_Price
0,1,4.603383e+06,34,12266,374.258776
1,2,1.225458e+07,20650,15144,342.363948
2,3,1.249095e+07,19569,16621,345.149344
3,4,1.462833e+07,23663,22263,318.519566
4,5,1.843183e+07,26665,27069,343.019950
5,6,1.400810e+07,23602,12274,390.458889
6,7,1.134207e+07,15043,19534,328.023420
7,8,9.640907e+06,18608,14961,287.196716
8,9,1.143355e+07,18855,17394,315.417076
9,10,1.720963e+07,26013,21112,365.191192


In [541]:
df = demand.drop(['Total_Price'], axis=1)

In [542]:
df

,Delivery Month,Advance_Order,Urgent_Order,Average_Price
0,1,34,12266,374.258776
1,2,20650,15144,342.363948
2,3,19569,16621,345.149344
3,4,23663,22263,318.519566
4,5,26665,27069,343.019950
5,6,23602,12274,390.458889
6,7,15043,19534,328.023420
7,8,18608,14961,287.196716
8,9,18855,17394,315.417076
9,10,26013,21112,365.191192


In [543]:
df.set_index('Delivery Month', inplace=True)

In [544]:
df

,Advance_Order,Urgent_Order,Average_Price
Delivery Month,,,
1,34,12266,374.258776
2,20650,15144,342.363948
3,19569,16621,345.149344
4,23663,22263,318.519566
5,26665,27069,343.019950
6,23602,12274,390.458889
7,15043,19534,328.023420
8,18608,14961,287.196716
9,18855,17394,315.417076


In [545]:
df['Predict_Demand'] = df['Advance_Order'] + df['Urgent_Order']

In [546]:
df

,Advance_Order,Urgent_Order,Average_Price,Predict_Demand
Delivery Month,,,,
1,34,12266,374.258776,12300
2,20650,15144,342.363948,35794
3,19569,16621,345.149344,36190
4,23663,22263,318.519566,45926
5,26665,27069,343.019950,53734
6,23602,12274,390.458889,35876
7,15043,19534,328.023420,34577
8,18608,14961,287.196716,33569
9,18855,17394,315.417076,36249


In [547]:
df_demand = df.drop(['Urgent_Order'], axis=1)

In [548]:
df_demand

,Advance_Order,Average_Price,Predict_Demand
Delivery Month,,,
1,34,374.258776,12300
2,20650,342.363948,35794
3,19569,345.149344,36190
4,23663,318.519566,45926
5,26665,343.019950,53734
6,23602,390.458889,35876
7,15043,328.023420,34577
8,18608,287.196716,33569
9,18855,315.417076,36249


In [549]:
train_size = int(len(df_demand) * 0.8)  # 80-20 split
train, test = df_demand[:train_size], df_demand[train_size:]

In [550]:
test 

,Advance_Order,Average_Price,Predict_Demand
Delivery Month,,,
21,20013,338.344682,35788
22,21267,315.187877,42145
23,23235,342.101706,42896
24,20768,341.443347,35586
25,16430,341.661775,32905
26,273,217.184268,273


In [551]:
order = (1, 1, 1)
seasonal_order = (1, 1, 1, 12)
exog_vars = ['Advance_Order', 'Average_Price']

In [552]:
stepwise_fit = auto_arima(train['Predict_Demand'], exogenous=train[exog_vars], start_p=1, start_q=1,
                          max_p=3, max_q=3, m=12, seasonal=True, stepwise=True, suppress_warnings=True,
                          error_action='ignore', scoring='mse')

c:\Users\tn_ad\AppData\Local\Programs\Python\Python311\Lib\site-packages\pmdarima\arima\utils.py:117: UserWarning: Appropriate D value may not have been reached; length of seasonally-differenced array (8) is shorter than m (12). Using D=1
  warnings.warn("Appropriate D value may not have been reached; "


In [553]:
best_params = stepwise_fit.get_params()
best_params

{'maxiter': 50,
 'method': 'lbfgs',
 'order': (0, 0, 1),
 'out_of_sample_size': 0,
 'scoring': 'mse',
 'scoring_args': {},
 'seasonal_order': (0, 1, 1, 12),
 'start_params': None,
 'suppress_warnings': True,
 'trend': None,
 'with_intercept': True}

In [554]:
sarimax_model = sm.tsa.statespace.SARIMAX(train['Predict_Demand'], exog=train[exog_vars],
                                         order=(best_params['order']), seasonal_order=(best_params['seasonal_order']))
sarimax_result = sarimax_model.fit(disp=False)

c:\Users\tn_ad\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\tn_ad\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\tn_ad\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\tn_ad\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn(

In [555]:
test

,Advance_Order,Average_Price,Predict_Demand
Delivery Month,,,
21,20013,338.344682,35788
22,21267,315.187877,42145
23,23235,342.101706,42896
24,20768,341.443347,35586
25,16430,341.661775,32905
26,273,217.184268,273


In [556]:
forecast = sarimax_result.get_forecast(steps=len(test), exog=[test[exog_vars]])
forecast_mean = forecast.predicted_mean

c:\Users\tn_ad\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\tn_ad\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [557]:
predict = forecast_mean.astype(int)
predict

20    39249
21    39049
22    44068
23    40288
24    33539
25     3730
Name: predicted_mean, dtype: int32

In [558]:
true = test['Predict_Demand']
true 

Delivery Month
21    35788
22    42145
23    42896
24    35586
25    32905
26      273
Name: Predict_Demand, dtype: int32

In [559]:
mse = mean_squared_error(true, predict)
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse}')

Root Mean Squared Error (RMSE): 3092.9740272645895


In [560]:
from sklearn.ensemble import RandomForestRegressor

In [561]:
X_train = train[['Average_Price', 'Advance_Order']]
y_train = train['Predict_Demand']

In [562]:
X_test = test[['Average_Price', 'Advance_Order']]
Y_test = test['Predict_Demand']

In [563]:
rf_model = RandomForestRegressor(n_estimators=50)
rf_model.fit(X_train,y_train)

RandomForestRegressor(n_estimators=50)

In [564]:
pred = rf_model.predict(X_test)

In [565]:
np.sqrt(mean_squared_error(pred,Y_test))

8258.609040009906

## Predict the demand for month 27

In [566]:
last_advance_order_value = df_demand['Advance_Order'].iloc[-1]

last_average_price_value = df_demand['Average_Price'].iloc[-1]

In [567]:
last_advance_order_value

273

In [568]:
last_average_price_value

217.18426787125435

In [569]:
exog_month27 = pd.DataFrame({'Advance Order': [last_advance_order_value], 'Average_Price': [last_average_price_value]})

In [570]:
forecast_month27 = sarimax_result.get_forecast(steps=1, exog=exog_month27)
print(f'Predicted Demand for Month 27: {forecast_month27.predicted_mean.astype(int).values[0]}')

Predicted Demand for Month 27: 8279


c:\Users\tn_ad\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\tn_ad\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


Use the average price and predicted demand to determine the probablities of each product type

In [571]:
def months_to_month_of_year(incremental_month):
    # Assuming January is the starting month (1-based index)
    starting_month = 1
    
    # Calculate the month of the year
    month_of_year = (starting_month + incremental_month - 1) % 12
    
    return month_of_year

In [572]:
incremental_month_value = 27
result_month = months_to_month_of_year(incremental_month_value)

print(f"{incremental_month_value} months is equivalent to month {result_month}")

27 months is equivalent to month 3


In [573]:
X_27month = pd.DataFrame({'MOY_Requested Delivery Date': [result_month], 'Items': [forecast_month27.predicted_mean.astype(int).values[0]], 'Average_Price': [last_average_price_value]})

In [574]:
X_27month['const'] = 1    
X_27month = X_27month[['const'] + [col for col in X_27month.columns if col != 'const']]
X_27month

,const,MOY_Requested Delivery Date,Items,Average_Price
0,1,3,8279,217.184268


In [575]:
X_27month_scaled = scaler.transform(X_27month)

In [576]:
Y_prob_27month = result.predict(X_27month_scaled)
Y_prob_27month

array([[0.34082952, 0.32041114, 0.33875934]])

Cacluate the predicted demand for each product type for month 27

In [577]:
prob_product1 = Y_prob_27month[0, 0]
prob_product2 = Y_prob_27month[0, 1]
prob_product3 = Y_prob_27month[0, 2]

In [578]:
Product1 = (forecast_month27.predicted_mean.values[0] * prob_product1).astype(int)
Product2 = (forecast_month27.predicted_mean.values[0] * prob_product2).astype(int)
Product3 = (forecast_month27.predicted_mean.values[0] * prob_product3).astype(int)

In [579]:
result_table = pd.DataFrame({
    'Product1': [Product1],
    'Product2': [Product2],
    'Product3': [Product3]
}, index=['Predicted_demand'])

print(result_table)

                  Product1  Product2  Product3
Predicted_demand      2821      2652      2804
